In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
import torch

from peft import PeftModel, PeftConfig

base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=False,
                                           trust_remote_code=True,
                                           add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)
englishFT = PeftModel.from_pretrained(base_model,"/home/spalaniappan9/nlp/project/weights/finetunedModel_englishData/checkpoint-8002")
tamilFT = PeftModel.from_pretrained(base_model,"/home/spalaniappan9/nlp/project/weights/finetunedModel/checkpoint-6135")


baseModel = base_model
englishModel = englishFT
tamilModel = tamilFT

tokenizer.pad_token = tokenizer.eos_token

/home/spalaniappan9/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/spalaniappan9/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:21<00:00,  5.38s/it]


In [2]:
import numpy as np

In [ ]:
def calculate_perplexity(model, test_sentences, tokenizer):
    total_log_prob = 0
    total_tokens = 0
    epsilon = 1e-10

    counter = 0
    for sentence in test_sentences:
        input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]

        sentence_log_prob = 0
        num_tokens = input_ids.size(1) - 1

        with torch.no_grad():
            for i in range(1, input_ids.size(1)): 
                context_ids = input_ids[:, :i]

                outputs = model(input_ids=context_ids)
                logits = outputs.logits
                last_logits = logits[0, -1, :] 

                next_token_id = input_ids[0, i]
                token_prob = torch.softmax(last_logits, dim=-1)[next_token_id].item()

                sentence_log_prob += np.log(token_prob + epsilon)

        total_log_prob += sentence_log_prob
        total_tokens += num_tokens
        counter += 1
        print(counter)
    
    avg_log_prob = total_log_prob / total_tokens

    perplexity = np.exp(-avg_log_prob)
    return perplexity

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

file_path = "./Application_programming_interface.txt"

with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

sentences = sent_tokenize(text)

print(len(sentences))

sentences_tamil = []

file_path = "./Application_programming_interface_tamil.txt"

with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()

sentences_tamil = sent_tokenize(text)
print(len(sentences_tamil))


144
149


[nltk_data] Downloading package punkt to
[nltk_data]     /home/spalaniappan9/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(calculate_perplexity(model=tamilModel, test_sentences=sentences_tamil, tokenizer=tokenizer))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
1.8081290894600317
